# Practical 6: Machine Learning for Power Systems

Instructors:
- Oluwatomisin Dada, University of Cambridge
- Cedric Kiplimo, Dedan Kimathi University of Technology

15-09-2025

**Abstract**: In this lab session, we will use a power system dataset to train two models (a linear model and a neural network) to solve an optimisation problem known as the alternating current optimal power flow (ACOPF), a rather difficult non-linear and non-convex optimisation problem.

# Primer: Power Systems
NOTE: You don't have to read this primer, although it is helpful to do so. If you wish to skip it, feel free to jump to [Section 7](#7-the-ieee-30-bus-test-case).

Before getting into the machine learning aspects, we shall first get a basic introduction to power systems that will help us understand the dataset better.

## 1. What is a Power System?
A **power system** is a network that produces, transports, and consumes alternating-current (AC) electrical energy. Think of it as the complex system that comprises generation, transmission, distribution, and consumption of electrical power. It can be modelled as a graph where nodes are the _buses_ and edges are the _lines/branches_. Below is a sample graph representing a power system.

<center>
    <img src="https://invenia.github.io/blog/public/images/power_grid_graphs.png">
</center>

Let's define some terms.

**Bus**  
A _bus_ is simply a network node (a graph vertex) where equipment connects and where we measure or impose voltages and injections. Here are some examples:
- Bus G: a generator terminal (power plant tied to the network).
- Bus S: a substation feeding a neighborhood.
- Bus L: a distribution node where many customer loads connect.

At a bus we typically care about voltage (magnitude and *phase*) and how much power is being injected or withdrawn there.

**Phase**  
AC voltages and currents are sinusoids. A _phase_ refers to one such sinusoidal waveform. Many power systems use **three-phase** (three sinusoids separated by 120°)—that’s how large systems deliver power efficiently. When electrical engineers analyse power systems, they formulate equations known as power flow equations.

For conceptual power-flow equations, they normally analyse one representative phase (single-phase equivalent) because steady-state three-phase, balanced systems reduce to three identical single-phase problems shifted in time. So when we talk about a voltage $V=V\angle\theta$  we refer to the magnitude and the _phase angle_ $\theta$ (the time shift of the sinusoid).


## 2. Representing Voltages and Currents as Phasors
A steady sinusoidal voltage is
$$v(t)=V_m \cos⁡(ωt+θ)$$

where $V_m$​ is amplitude, $\omega$ is angular frequency, and $\theta$ is the phase offset.

When everything is sinusoidal at the same $\omega$ (steady-state), we replace $v(t)$ by its phasor
$$\mathbf{V} = V e^{j\theta}$$
where VVV commonly denotes RMS amplitude (engineer’s convention) and θ\thetaθ is the phase angle. Similarly current $i(t)$ $\leftrightarrow$ phasor $\mathbf{I}=I e^{j\phi}$.

By doing this, differentiation/integration become algebraic multiplications by $j\omega$, and Ohm’s/Kirchhoff’s laws become _algebraic equations in the complex domain_. This converts time-domain differential equations into linear algebra with complex numbers — huge simplification.

## 3. AC Power is Complex
A **port** is a terminal or pair of terminals through which a device exchanges electrical power with the network. Practically, a generator terminal is a port (the place it injects current/voltage to the grid).

**Complex power** $S$ at a port is defined as $S=P+jQS$.
    - $P$ (real/active power) is the energy per second doing useful work (watts).
    - $Q$ (reactive power) is the power that oscillates back and forth because of inductors/capacitors — it doesn’t do long-term work but it affects voltages and currents.

In phasor form we use:  
  $$S=VI^*$$
  where $V$ and $I$ are complex phasors and $I^*$ is the complex conjugate of the current phasor. This convention gives $P$ and $Q$ values that are independent of the arbitrary reference angle.

## 4. Real, Reactive, and Apparent power
Think of the grid as a big circuit built from resistors (R), inductors (L), and capacitors (C). These elements create different relationships between voltage and current:
- **Resistor (R):** voltage and current are _in phase_. Energy is dissipated as heat → this is _real_ (active) power $P$.
- **Inductor/Capacitor (L/C):** voltage and current are _out of phase_. Energy is alternately stored in fields and returned to the network → _reactive_ behavior $Q$. Over a cycle, ideal L/C do not consume net energy.

**Formal quantities:**
- **Complex power (apparent):** $S=P+jQS$.
    - $P$ = real/active power (watts) = average power delivered/consumed.
    - $Q$ = reactive power (vars) = oscillatory exchange due to L/C.
    
- **Apparent power magnitude:** $∣S∣= V_\text{rms} I_\text{rms}$.
    
- **Power factor:** $\text{PF} = P / |S| = \cos(\phi)$, where $\phi$ is the phase difference between voltage and current phasors.

**Why reactive power matters operationally:**  
Reactive power does not perform useful work but increases currents for the same real power. Higher currents lead to more $I^2 R$ losses, larger thermal stress on lines and transformers, and difficulty maintaining acceptable voltages. Operators must manage reactive power to keep voltages in safe ranges and limit losses.

## 5. Power Flow Expressions
### Part 1: The Simple Case
Consider two buses $i$ and $j$ connected by a single series line. The quantity $p_{ij}$​ denotes the _real power flowing on that branch from bus $i$ toward bus $j$_ (positive means power leaves $i$ toward $j$). Similarly $q_{ij}$​ is the reactive power on that same directed branch.

Let
- $V_i = v_i e^{j\theta_i}$ and $V_i = v_i e^{j\theta_j}$ be the complex phasor voltages at the two buses (magnitudes $v$, angles $\theta$).
- The line series admittance is $y_{ij} = g_{ij} + j b_{ij}$ (the inverse of the line impedance $z_{ij}=r_{ij}+j x_{ij}$).
- The angle difference $\delta_{ij}:=\theta_i - \theta_j$

Then, the current from $i$ to $j$  is
$$I_{ij}=y_{ij}​(V_i​ − V_j​)$$
The complex power at bus $i$ into the branch is
$$
\begin{align}
	S_{ij} &= V_i I^*_{ij} \\\\
	&= V_i (y^*_{ij}​(V^*_i​ − V^*_j​)) = v^2_i y^*_{ij} - v_i v_j e^{j\delta_{ij}}y^*_{ij}
\end{align}
$$
where $y^*_{ij} = g_{ij} - jb_{ij}$. We then take the real and imaginary parts to get the real and reactive branch powers, which are:
* **Real power on branch $i$ → $j$** :
  $$p_{ij} = v_i^2 g_{ij} - v_i v_j (g_{ij} \cos \delta_{ij} + b_{ij} \sin \delta_{ij})$$
* **Reactive on branch $i$ → $j$**:
  $$q_{ij} = -v_i^2 b_{ij} - v_i v_j (g_{ij} \sin \delta_{ij} - b_{ij} \cos \delta_{ij})$$

Note the following:
- The first term ($v_i^2 g_{ij}$ in $p_{ij}$ and $-v_i^2 b_{ij}$ in $q_{ij}$) are _local_ contributions (depend only on bus $i$ and the branch admittance).
- The second term contains $v_i v_j$​ and trig of the angle difference: these are _coupling_ terms between buses $i$ and $j$.

**Note the nonlinearity:** Terms like $v_i v_j \cos\delta_{ij}$ ​and $\sin\delta_{ij}$​ produce *products of magnitudes* and *trigonometric functions of angle differences*. Those are the nonlinear pieces that make AC power flow non-linear and non-convex.

*However*, if you set $v_i = v_j =1\;pu$ and assume small $\delta_{ij}$​ and small $r$ vs $x$, you can linearize these expressions (that leads to the DC approximation).

### Part 2: The General Case - Nodal Power Balance
At a given bus $i$, many branches (lines) can meet. The **net complex power injected** at bus $i$ (positive if injected by generation, negative if consumed by loads) equals the sum of the complex powers on all branches leaving $i$ plus any shunt injections at the bus. This is the (complex) nodal power balance — it enforces Kirchhoff’s current law and defines the power-flow equations we solve.

We end up with these equations:
- **Nodal real power injection at bus $i$:**
$$P_i \;=\; \sum_{j=1}^N v_i v_j\big(G_{ij}\cos\delta_{ij} + B_{ij}\sin\delta_{ij}\big)$$
- **Nodal reactive power injection at bus $j$:**
$$Q_i \;=\; \sum_{j=1}^N v_i v_j\big(G_{ij}\sin\delta_{ij} - B_{ij}\cos\delta_{ij}\big)$$

### Part 3: What are we solving?
We are given _specified_ injections at each bus (depending on bus type):

- For a **PQ bus**: specified ($P_i^\text{spec}$, $Q_i^\text{spec}$).
- For a **PV bus**: specified ($P_i^\text{spec}$, $Q_i^\text{spec}$) (reactive $Q_i$​ is unknown).
- For the **slack bus**: specified ($V_i^\text{spec}$, $\theta_i^\text{spec}$) (it absorbs any net mismatch).

The **computed** injections from the current voltage guess are $P_i^\text{calc}(V, \theta)$, $Q_i^\text{calc}(V, \theta)$) given by the nodal formulas above. They can be obtained by simulation e.g. on MATLAB Simulink.

The mismatch (power residuals) for each bus are:

for each bus:
$$
\begin{align}
	\Delta P_i(V,\theta) &\;=\; P_i^{\text{spec}} \;-\; P_i^{\text{calc}}(V,\theta)\\[10pt]
	\Delta Q_i(V,\theta) &\;=\; Q_i^{\text{spec}} \;-\; Q_i^{\text{calc}}(V,\theta)
\end{align}
$$

Collect the relevant mismatches into a vector $F(x)$ where $x$ stacks the **unknown** voltage magnitudes and angles (the exact components of $x$ depend on bus types). For example, if all PQ buses have unknown $(v,\theta)$ and PV buses have unknown $\theta$, then

$$
F(x) = \begin{bmatrix}\text{(unknown angles)}\\[2pt]\text{(unknown magnitudes)}\end{bmatrix}, \qquad F(x) = \begin{bmatrix}\Delta P\\[2pt]\Delta Q\end{bmatrix}
$$
(with rows corresponding only to the equations that must be satisfied — e.g., no $\Delta Q$ row for a PV bus where $Q$ is not specified).

Concisely put, our **goal** is to *find x (unknown voltages/angles) such that $F(x)=0$* i.e., _find the voltage magnitudes and angles so that every bus’s power balance holds_.

## 6. Optimal Power Flow (OPF)
When we solve the power flow equations alone, we are simply finding voltages and currents consistent with physics. The only constraints we are following are physical ones. But in real power grids, we are concerned with more than just physics. For example, we might be interested in solving the power flow equations such that generator cost is minimised. This is especially useful in complex power grids where there many generators generating power at different costs. Our desire would be to *solve the power flow equations such that both the physical constraints and economic constraints are satisfied*.

This is known as optimal power flow (OPF). The goal of OPF is to choose control variables (generator outputs, tap settings, etc.) to minimize an objective (cost, losses, whatever) subject to the physical power-flow equations and operational limits. To the power flow equations, we add:
- Decision variables: generator real power $P_G$, reactive power $Q_G$, and voltage magnitudes $V$.
- Objective function e.g., minimise cost
- Constraints:
	- PF equations must hold
	- Generator limits ($P^{min}_G$, $P^{max}_G$ )
	- Voltage limits ($V^{min}_G$, $V^{max}_G$ )
	- Line limits (flows can't exceed ratings)

In short, OPF says *choose generator setpoints and voltages that satisfy physics and minimize some objective*.

### Part 1: AC-OPF
Here we use the **full nonlinear PF equations** and solve it subject to chosen constraints.

$$
\begin{aligned} \min_{P_G, Q_G, V, \theta} & \quad \sum_{g \in \text{generators}} C_g(P_{Gg}) \\ \text{s.t.} & \quad P_i = P_{Gi} - P_{Li}, \\ & \quad Q_i = Q_{Gi} - Q_{Li}, \\ & \quad \text{(PF equations hold for all buses)} \\ & \quad P_{G}^{\min} \le P_G \le P_{G}^{\max}, \\ & \quad V^{\min} \le V \le V^{\max}, \\ & \quad |S_{ij}| \le S_{ij}^{\max}. \end{aligned}
$$
- $P_{Li}$, $Q_{Li}$: load demand at bus $i$
- $C_g(\cdot)$: generator cost (often quadratic)
- $S_{ij}$: apparent power flow on line $i$-$j$

AC-OPF is accurate but hard. The equalities are nonlinear and nonconvex (products $v_i v_j$​ and trig of angle differences). It is therefore a nonconvex constrained nonlinear program.

### Part 2: DC-OPF
To make OPF tractable, we simplify it as follows:
- Assume all voltages are ~$1$ p.u. (ignore magnitude variation)
- Ignore reactive power
- Assume lines are mostly inductive (resistance $\approx 0$)

Then:
$$P_{ij} = \frac{1}{X_{ij}} (\theta_i - \theta_j)$$
where $X_{ij}$ is line reactance.

The nodal balance becomes:
$$P_i = P_{Gi} - P_{Li} = \sum_i \frac{1}{X_{ij}} (\theta_i - \theta_j)$$
which is linear system - much easier to solve.

$$
\begin{aligned}
\min_{P_G, \theta} & \quad \sum_{g \in \text{generators}} C_g(P_{Gg}) \\ \text{s.t.} & \quad P_{Gi} - P_{Li} = \sum_j \tfrac{1}{X_{ij}}(\theta_i - \theta_j), \\ & \quad P_G^{\min} \le P_G \le P_G^{\max}, \\ & \quad |P_{ij}| \le P_{ij}^{\max}
\end{aligned}
$$

DCOPF is much faster to solve because it is linear (or convex quadratic). It is widely used in day-to-day market operations but is less accurate because it ignores voltage/reactive effects.

## 7. The IEEE 30-bus Test Case
It’s a **benchmark power system** model, originally published by the IEEE (Institute of Electrical and Electronics Engineers), that represents a *small but realistic transmission grid*. It is a simple approximation of a section of the American power grid as it was in 1961. Researchers and engineers use it to:
- Test **power flow algorithms**.
- Validate and compare **ACOPF/DCOPF solvers**.
- Benchmark **optimization, machine learning, or control methods**.

### Its Components
This benchmark system comprises:
- **30 buses (nodes)**: These are connection points — either loads, generators, or junctions.
- **41 transmission lines/branches**: They connect buses, each with reactance, resistance, and line limits.
- **6 generators**: Located at specific buses (e.g., bus 1 is the slack/reference bus). They supply real and reactive power within limits.
- **Loads**: Many buses represent demand, specified in MW (real power) and Mvar (reactive power).
- **Shunts**: Some buses include shunt elements (like capacitors) to support voltage.

So it can be pictured as a **graph** with 30 nodes and 41 edges, but where each node has electrical variables (voltage, angle, load, generation).

### Simulating the IEEE 30-bus Test Case on MATLAB
This benchmark system can be simulated on MATLAB using MATPOWER, an open-source MATLAB/Octave package for **power system simulation and optimization**. Think of it as the **framework** (like PyTorch or TensorFlow) that loads the dataset and runs baseline solvers.

It’s widely used by researchers and utilities for:
- Solving **power flow (PF)** problems.
- Running **optimal power flow (OPF)** (both AC and DC).
- Benchmarking new algorithms.

Think of it as a “toolbox” where you load a test case (like IEEE 30-bus) and run solvers. Using MATPOWER, you can run PF, OPF (AC and DC) to solve the power flow equations

The dataset we will use in this practical is generated after running a simulation of the IEEE 30-bus test case in MATPOWER.

## 8. Understanding and Using the Dataset
In this lab, we shall use a dataset generated by solving AC-OPF of the IEEE 30-bus test case using MATPOWER. Our goal is to compare the performances of two machine learning models trained on this dataset at solving the AC-OPF problem. The simulation results were initially saved in the MATLAB `.mat` format but the code for extracting the data into familiar formats has been provided.

Let's get into it.

In [ ]:
! git clone https://github.com/kiplimock/ieee-case30-data.git
! pip install osmnx

In [ ]:

import sys
sys.path.append("/content/ieee-case30-data/fynesse_mlfc")
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [ ]:
from fynesse.access import extract_data

First we need to load our data. At first, our data is stored in a dictionary with three keys - `train`, `val`, and `test`, which store the training, validation, and  test data we will use for our models.

In [ ]:
data = extract_data(data_path = "/content/ieee-case30-data/matpower_format/case30", standardise_outputs=False)
keys = ['train', 'val', 'test']

Each of these three dictionaries have four arrays that can be accesses using four keys - `X`, `Y`, `A`, `B`. These four arrays are the model inputs, model outputs, network adjacency matrices, and bus susceptances, respectively. For this lab, we shall mostly be interested in the model inputs and ouputs because we will train our models using them. We will also use the DCOPF data.

In [ ]:
data[keys[0]].keys()

### Examining the Data
Before proceeding with any modelling, please take some time to understand the variables in the dataset by going through this [README](README.md) file. In particular, focus on the `X` and `Y` arrays. Try to gain some intutions for the meaning of the dataset.

#### Task 1
Focusing on the train set, examine the inputs `X`. Notice the shape of the array.

What do you think the various dimensions of the array correspond to?

(**Hint:** Think back to the IEEE 30-bus test case).

In [ ]:
X_train = data[keys[0]]['X']
X_train.shape

`X` train array has shape `(600, 30, 4)`.
- Dimension 0 - the number of samples
- Dimension 1 - the number of buses in the system
- Dimension 2 - the number of features in the dataset i.e. $P_d$, $Q_d$, `mag_Sd` and `ang_Sd`.

#### Task 2
Do the same for `Y`.

**Question**: What do the dimensions correspond to?

In [ ]:
# TODO
# Y_train = ?

#### Task 2 Extended
Look at the test and validation sets as well

In [ ]:
# TODO

### Some Basic Prediction Models
We will now use our data to see the performances of three basic solutions to the OPF problem:
- **Gridwise averaging approach**: Computes an average for outputs across the whole grid, leading to a single output for the entire grid. This average can be used as a baseline predictor for voltage and phase at a bus.
- **Nodewise averaging approach**: Computes an average for outputs per bus (node). So we begin with $n$ samples for each bus and aggregate those samples to end with $1$ for each bus. This average can be used as a baseline predictor for voltage and phase at a bus.
- **DC-OPF**: The `dcopf` results from the simulation. It is a linearised simplification (and thus an estimate) of the ACOPF.

Our comparison metrics will be mean absolute error (MAE), mean squared error (MSE), and fraction of variance unexplained (FVU).

### A. Gridwise Averaging

To  average our outputs across the entire grid, we compute the mean of the array for each of the outputs (`Vm` and `Va`). For each output, the averaging yields a scalar result. Because we would like to use this result as a basline predictor, we will need to expand its dimensions to match its original shape for comparison.

Notice how we are expanding our predictor output using `np.resize()`. Why did we do that? To understand why, check the shapes of `ym`, `np.resize(ym, data[key]['Y'][...,0].shape)`, and `data[key]['Y'][...,0]`. Do you notice that the shapes of `data[key]['Y'][...,0]` and `np.resize(ym, data[key]['Y'][...,0].shape)` are the same?

We do the resizing because, for us to easily compare two arrays together using regression metrics like MSE, they should be of the same shape so that residuals can be computed using vectorised operations. This is very important. We will ensure we do this for all other approaches we study in this lab.

In [ ]:
# Gridwise Averaging Approach --> average over entire grid
ym, ya = data[keys[0]]['Y'][...,0].mean(), data[keys[0]]['Y'][...,1].mean()

gridwise = {}
for key in keys:
    Vm_train = data[key]['Y'][...,0]
    Va_train = data[key]['Y'][...,1]
    gridwise.update({key : (np.resize(ym, Vm_train.shape), np.resize(ya, Va_train.shape))})

# err_gw = print_err(target=data, pred_dict=gridwise)

We saved our performance metrics in a dictionary called `gridwise`. The structure of this dictionary is important because we shall use it to store the metrics for all the "models" we shall use for prediction in this lab. This structure is as follows:
```python
{
    "train": (Vm_train_pred, Va_train_pred),
    "val": (Vm_val_pred, Va_val_pred),
    "test": (Vm_test_pred, Va_test_pred)
}
```

where `Vm_train_pred.shape = data[keys[0]]['Y'][...,0].shape` and `Va_train_pred.shape = data[keys[0]]['Y'][...,1].shape`. The same applies for `val` and `test`. Pay attention to this structure because you will need it when calculating the performance metrics in the next step.

Let's now calculate the regression metrics for the train, validation, and test sets.

In [ ]:
err_dict = {}
target = data.copy()
for key in gridwise.keys():
    complex_target = data[key]['Y'][:,:,0] * np.exp(1j * data[key]['Y'][:,:,1])
    V_pred = gridwise[key]
    V_target = target[key]['Y']
    # print(key, V_pred[0].shape, V_target[:,:,0].shape)

    if V_pred[0].shape == V_target[:,:,0].shape:
        mag_err = V_pred[0] - V_target[:,:,0] # Vm
        ang_err = V_pred[1] - V_target[:,:,1] # Va
        complex_pred = gridwise[key][0] * np.exp(1j * gridwise[key][1])

    else:
        print("Shape mismatch! Broadcasting ...")
        mag_err = gridwise[key][0][0,:] - target[key]['Y'][:,:,0] # Vm
        ang_err = gridwise[key][1][0:,] - target[key]['Y'][:,:,1] # Va
        complex_pred = gridwise[key][0][0:,] * np.exp(1j * gridwise[key][1][0,:])

    complex_err =  complex_pred - complex_target

    print(f"\n{key.capitalize()} set metrics:")
    print(f"Vm MAE: {np.abs(mag_err).mean():.5e}")
    print(f"Vm MSE: {np.square(mag_err).mean():.5e}")
    print(f"Vm FVU: {(np.square(mag_err).mean() / V_target[:,:,0].var()):.5e}")

    print("")

    print(f"Va MAE: {np.abs(ang_err).mean():.5e}")
    print(f"Va MSE: {np.square(ang_err).mean():.5e}")
    print(f"Va FVU: {(np.square(ang_err).mean() / V_target[:,:,0].var()):.5e}")


#### Task 3 - Function to Calculate Metrics
For reusability, create a function `print_err` to display the metrics and save them for later use. Most of the code for this has already been provided. You should add this function to you Access Assess Address pipeline.

In [ ]:
def print_err(target, pred_dict):
    """
    Prints error metrics between target and predictions.
    Args:
        target (dict): Dictionary containing true values with keys 'train', 'val', 'test'.
                       Each key maps to another dict with keys 'X', 'Y', 'A', and 'B'.
        pred_dict (dict): Dictionary containing predicted values with keys 'train', 'val', 'test'.
                          Each key maps to a tuple of (predicted_magnitudes, predicted_angles).

    Returns:
        err_dict (dict): Dictionary containing error metrics for each dataset split.
    """
    err_dict = {} # for saving the metrics
    for key in pred_dict.keys():
        pass
        # TODO

        # 1. Calculate the error in predicted magnitudes and angles
        # 2. Also calculate the error in complex form for completeness
        # 3. Compute and print MAE, MSE, and FVU for each of these errors
        # 4. Save and return these metrics in a dictionary for later use

    # return err_dict

    raise NotImplementedError("Function logic not yet implementd.")

### B. Nodewise Averaging

We average the outputs (`Vm` and `Va`) per bus/node. We begin with a `(n_samples x n_buses x n_outputs)` array for the entire system and end up with a `(n_buses x n_outputs)` array i.e. a vectors of size n_buses for each of the outputs.

Let's try to think about how should perform this  averaging. We know that our arrays have shape `(n_samples, n_buses, n_outputs)`. This tells us that at each bus/node, we are measuring `n_output` variables `n_samples` times. Nodewise averaging means that for each of the variables, we average all the samples taken so that we end up with only one value at each node for each varible. For example, at bus/node 1, we have one value for var_1 and one for var_2. This is what we mean when we say we begin with a `(n_samples x n_buses x n_outputs)` array and end up with a `(n_buses x n_outputs)` array. Think about this can be implemented in NumPy.

Afterwards, we will use the function we just created to compute the metrics of this baseline model.

#### Task 4 - Implement Nodewise Averaging
Now you have all you need to implement nodewise averaging.

In [ ]:
# TODO
# Nodewise Averaging Approach --> average per node
# The code for computing ym and ya is very similar to the gridwise approach, but you need to make a small change. Think about what that change is.

err_nw = None

### C. DC-OPF

We treat the DCOPF output of the simulation as a baseline model and compute the metrics. But first we need to access the DCOPF outputs from the simulation. We can do this by first accessing the array saved in the susceptance (`B`) key of `data[key]` dictionary e.g., `data['train]['B']` or `data['val']['B']`. This gives an array of length `n_samples` where each element is a dictionary with multiple keys, one of which is `V-dcopf` which we are interested in.

So we can read one sample of DCOPF using `data['train']['B'][i]['V-dcopf']` for some `i`. Each sample, as you might guess is of shape `(n_buses, n_outputs)`, which in our case is (30, 2). You can see how this completes the picture because `n_samples` samples of shape `(n_buses, n_outputs)` makes an aggregate array of shape `(n_samples, n_buses, n_outputs)`. Now all we will need to do is write some clever code for accessing the DCOPF of each sample and then stack them together in a single array. Of course we will need to this for each set of data i.e., train, test, and validate.

#### Task 5 - Check Performance of DC-OPF
The code for preparing the data has been provided for you here so that you can focus on what you need to learn.

In [ ]:

dcopf = {}
for key in keys: # 'train', 'val', 'test'
    tmp = np.array([item['V-dcopf'] for item in data[key]['B']]) # --> [n_samples, n_buses, n_outputs]
    ym = tmp[..., 0]
    ya = np.pi * (tmp[..., 1]/180) # convert degrees to radians

    # TODO update the dcopf dictionary. Hint: look at how we did it for the gridwise and nodewise approaches.
    dcopf.update()
del tmp

err_dc = None

### D. Linear Model

Now we turn our attention to a more sophisticated model - linear regression. Our goal is to build a model to predict `Vm` and `Va` at each bus using only two out of the four features - `Pd` and `Qd`. Our input is 3-dimensional with shape `(n_samples, n_buses, n_features)`. We can merge the last two dimensions so that the new shape is `(n_samples, n_buses * n_features)` so that our input has `n_buses * n_features` columns that we can use as features for our linear regression model.

But there is a catch. Some buses are inactive i.e., there `Pd` and `Qd` values are zero. We should build our model with data from only active buses. So we'll drop readings from inactive buses. The new number of columns will be some $n \leq n_{buses} \times n_{features}$

Something else. It's important to understand what the model we are building actually means in the context of our IEEE 30-bus power system. The outputs `Vm` and `Va` are measured at each bus. We want to predict these values at each bus too, and we are using the `Pd` and `Qd` measurements from all active buses in the grid to build our model. That means we will have one model to predict each output at each of the buses i.e., `n_buses` models to predict `Vm` and another `n_buses` to predict `Va`.

#### Task 6 - Write a Function to Train Linear Regression Models
Write the function `train_lm` to train and save these models.

In [ ]:
inp = data[keys[0]]['X'][:,:,:2] # --> (n_samples, n_buses, n_features)
out = data[keys[0]]['Y'] # --> (n_samples, n_buses, n_outputs)

def train_lm(inp, out, model=LinearRegression):
    """
    Trains linear regression models for each bus/node in the power system.

    Args:
        inp (np.ndarray): Input features of shape (n_samples, n_buses, n_features).
        out (np.ndarray): Output targets of shape (n_samples, n_buses, n_outputs).
        model (class): Linear regression model class from sklearn. Default is LinearRegression.

    Returns:
        mag_models (list): List of trained models for voltage magnitudes (Vm) at each bus.
        ang_models (list): List of trained models for voltage angles (Va) at each bus
    """

    model = LinearRegression

    node_count = out.shape[1]
    mag_models, ang_models = [], []

    # TODO
    # 1. Find index of nodes with nonzero loads and use as input to model
    nonzero_indx = np.abs(inp[:,:,:2].mean(0)).sum(1) != 0 # boolean index of nonzero (active) load buses

    # 2. Reshape the input to shape (n_samples, n_features*n_nonzero_buses). Use `nonzero_indx` as a mask to filter active buses
    # 3. Train the models

    # return mag_models, ang_models

    raise NotImplementedError("Function logic not yet implementd.")

# train_lm(inp, out)

We've written a helper function for you to obtain the predictions from the linear models.

In [ ]:
def get_predictions(models, inp):
    shape = inp.shape[:2]
    ym, ya = np.zeros(shape), np.zeros(shape)
    nonzero_indx = np.abs(inp[:,:,:2].mean(0)).sum(1) != 0
    inp_reshaped = inp[:,nonzero_indx,:2].reshape(-1, nonzero_indx.sum()*2) # --> (samples, n_features*n_nonzero_buses)

    for i in range(shape[1]): # prediction for each bus yields array of shape [n_samples,]
        ym[:,i] = models[0][i].predict(inp_reshaped)
        ya[:,i] = models[1][i].predict(inp_reshaped)

    return (ym, ya)

#### Task 7 - Obtain Predictions of the Linear Model and Store Them
Using the helper function provide, obtain the linear model's predictions and store them in a dictionary as you did with the other models.

In [ ]:
# Linear Model
m_mod, a_mod = train_lm(data[keys[0]]['X'], data[keys[0]]['Y'], LinearRegression)
lm = {}

# TODO
# store predictions in lm dictionary

err_lm = print_err(target=data, pred_dict=lm)

The metrics of all our models have been saved in dictionaries. Now we need to visualise them. Try to think about the performance of the four models we have used so far - gridwise, averaging, nodewise averaging, DCOPF, and linear regression. Which do you think has the best performance and which one has the best performance? Think about how those models are built and make an educated guess about how they performed. Rank them in descending order of performance, say based on MSE.

In [ ]:
# TODO
# Rank the models based on MSE (your guess). Also discuss why you think they will perform the way you think?

Now let's visualise the metrics. We begin with `MSE`, and then proceed to the rest.

In [ ]:
errors = [err_gw, err_nw, err_dc, err_lm]
err_key = "MSE"
methods=['grid', 'node', 'dcopf', 'lm']

show_mag, show_ang, log, sel_key = True, True, True, None
rows = len(errors[0].keys()) if sel_key is None else len(sel_key)
cols = int(show_mag) + int(show_ang)

if err_key == "MCEM":
    cols = 1
    show_mag, show_ang = False, False
if cols == 0:
    raise ValueError("No columns in the figure")

fig, axs = plt.subplots(rows, cols, sharex=True)
bar_colors = [
    'tab:blue',
    'tab:orange',
    'tab:green',
    'tab:red',
    'tab:purple',
    'tab:brown'
    'tab:pink',
    'tab:gray',
    'tab:olive'
    'tab:cyan',
] * 5

sel_key = list(errors[0].keys()) if sel_key is None else sel_key

if rows == 1 and cols == 1:
    sel_col = 'Vm' if show_mag else 'Va'
    col_key = sel_col + ' ' + err_key
    if err_key == "MCEM":
        sel_col = 'Mean Complex Err Mag'
        col_key = err_key
    counts = [error[sel_key[0]][col_key] for error in errors]
    if log:
        axs.set_yscale('log')
    axs.bar(methods, counts, color=bar_colors[:len(counts)])
    axs.set_ylabel(err_key)
    axs.set_title(sel_col)

elif rows > 1:
    for i, j in enumerate(sel_key):
        if show_mag and show_ang:
            counts = [error[j]['Vm ' + err_key] for error in errors]
            if log:
                axs[i, 0].set_yscale('log')
                axs[i, 1].set_yscale('log')
            axs[i, 0].bar(methods, counts, color=bar_colors[:len(counts)])
            axs[i, 0].set_ylabel(err_key)
            if i == 0:
                axs[i, 0].set_title('Vm')

            counts = [error[j]['Va ' + err_key] for error in errors]
            axs[i, 1].bar(methods, counts, color=bar_colors[:len(counts)])
            if i == 0:
                axs[i, 1].set_title('Va')

        else:
            sel_col = 'Vm' if show_mag else 'Va'
            col_key = sel_col+' '+err_key
            if err_key == 'MCEM':
                sel_col = 'Mean Complex Err Mag.'
                col_key = err_key
            counts = [error[sel_key[0]][col_key] for error in errors]
            if log:
                axs[i].set_yscale('log')
            axs[i].bar(methods, counts, color=bar_colors[:len(counts)])
            axs[i].set_ylabel(err_key)
            if i == 0:
                axs[i].set_title(sel_col)

For reusability, create a function `plot_comparison_bar` that we will use afterwards for plotting the comparison between the various models for a particular metric. The code for this function is already provided in the previous code cell, but you will need to make minor modifications to make it reusable.

In [ ]:
def plot_comparison_bar(errors, err_key, methods, sel_key=None, *, show_mag=True, show_ang=True, logy=True):
    """
    Plots a comparison bar chart for different error metrics across various methods.
    """
    rows = len(errors[0].keys()) if sel_key is None else len(sel_key)
    cols = int(show_mag)+int(show_ang)

    # TODO
    # Complete the function logic here. You can refer to the code above for guidance.

    raise NotImplementedError("Function logic not yet implementd.")

Using the function you just created, compete the four models you've used so far in terms of:
1. FVU

In [ ]:
plot_comparison_bar(errors=[err_gw, err_nw, err_dc, err_lm], err_key='FVU', methods=['grid', 'node', 'dcopf', 'lm'])

2. MAE

In [ ]:
plot_comparison_bar(errors=[err_gw, err_nw, err_dc, err_lm], err_key='MAE', methods=['grid', 'node', 'dcopf', 'lm'])

3. MCEM

In [ ]:
plot_comparison_bar(errors=[err_gw, err_nw, err_dc, err_lm], err_key='MCEM', methods=['grid', 'node', 'dcopf', 'lm'])

As you can see, for all the metrics, the worst performance was by the gridwise averging and DCOPF. This was expected because these models are too simple and do not capture the complexity of the grid. One averages across the entire grid while the other linearises a non-linear non-convex optimisation problem. Nodewise averaging was better than those two, but still not really good enough. Although averaging by bus captures more complexity, it's still a simple averaging operation. Linear regression is the best model by far. Not at all suprising because we might expect that there is a linear relationship between power (`Pd` and `Qd`) and voltages (`Vm` and `Va`).

### E. Neural Network
Now we move on to something complex - neural networks. NNs often succeed where simpler models fail because of their ability to learn the non-linear relationships between various variables in a dataset. In this case, we shall train a simple fully-connected multi-layer perceptron (MLP).

Try to think about how we should build the MLP.

How many input and output "neurons" should we have?

For the case of our dataset, do you expect the NN to do better than the linear model?

In [ ]:
# TODO
# Size of input and output layers of the MLP
# Performance of the MLP compared to the linear model

In [ ]:
import tqdm
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

from fynesse.access import prep_loaders

#### Task 8 - Define a class for an MLP
Your task is to build an MLP with a hidden layer of 1024 units. The number of units in the input and output layers is determined by the nature of your data. If you answered the previous questions about the MLP correctly, you should be good to go here.

The important thing to remember about building NNs in PyTorch is that your class should inherit from `nn.Module` and you must define two methods in your class - an `__init__` method where you define your players and a `forward` method where your forward pass operation is performed.

Once your class definition is fine, consider adding it to your Access Assess Address framework. Where in the framework do you think it should go?

In [ ]:
# MLP class definition
class MLP(nn.Module):
    def __init__(self, ):
        pass
        # TODO
        # Define the layers of the MLP here

    def forward(self, x):
        pass
        # TODO
        # Define the forward pass

        raise NotImplementedError("Function logic not yet implementd.")

In case you are new to PyTorch, we have defined a helper function `prep_loaders` for you to load your dataset and convert them into tensors so that we can take advantage of PyTorch's automatic differentiation utility and also train our model faster on a GPU.

In [ ]:
from fynesse.access import prep_loaders

Now we need to prepare our data using this function to make them ready for passing into the NN. It is always good practice to scale your data before using them to train any model.

In [ ]:
inp0, out0 = data[keys[0]]['X'], data[keys[0]]['Y']

num_nodes = inp0.shape[1]
num_samples = inp0.shape[0]

# Normalize features
scaler_inp = StandardScaler()
scaler_out = StandardScaler()

# Nonzero mask
bool_nonzero_indx = (inp0[:,:,0] + inp0[:,:,1]).mean(0) != 0

# Reshape from 3D to 2D --> [n_samples, n_buses*n_features]
inp_shape = (num_samples, int(2*bool_nonzero_indx.sum()))

# Select only 2 features (Pd & Qd) and nonzero load nodes
sel_inp0 = inp0[:, bool_nonzero_indx, :2].reshape(inp_shape)
sel_out0 = out0.reshape((num_samples, num_nodes*2))

# scale the data
scaler_inp.fit(sel_inp0)
scaler_out.fit(sel_out0)

# convert the data to tensors
loaders = prep_loaders(data=data, keys=keys, scaler_inp=scaler_inp, scaler_out=scaler_out,)

Now it's time to train our NN using our scaled data. We will train our MLP in 200 epochs (feel free to experiment with fewer or more epochs). When training models in PyTorch, we need to provide the objective function (known as a criterion) and the algorithm for doing backward propagation (known as an optimiser). There are other quirks for training models with PyTorch, like disabling gradient calculation when running an inference on the model because we are not doing any training. There are many others, but there will suffice for now.

We will track the training and validation losses while the NN is training so that we can visualise them later.It is possible to visualise the progress of training live by using [TensorBoard](https://docs.pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html), but this is outside the scope of this practical. For now, we shall just save the losses during the training and then visualise afterwards.

In [ ]:
c_in, c_out = int(2*bool_nonzero_indx.sum()), int(2*num_nodes)

# initialise model
model = MLP(c_in=c_in, c_hidden=1024, c_out=c_out, num_layers=5, dp_rate=0.1)

# specify loss fn and initialise optimiser
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


num_epochs = 200
train_losses = []
val_losses = []

for epoch in tqdm.trange(num_epochs):
    model.train()
    running_train_loss = 0.0
    for batch_x, batch_y in loaders[0]:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()

    # Validation
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for val_x, val_y in loaders[1]:
            val_outputs = model(val_x)
            val_loss = criterion(val_outputs, val_y)
            running_val_loss += val_loss.item()

    avg_train_loss = running_train_loss / len(loaders[0])
    avg_val_loss = running_val_loss / len(loaders[1])

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)


model.eval()
running_test_loss = 0.0
with torch.no_grad():
    for test_x, test_y in loaders[2]:
        test_outputs = model(test_x)
        test_loss = criterion(test_outputs, test_y)
        running_test_loss += test_loss.item()
    avg_test_loss = running_test_loss / len(loaders[2])
    print(f"\n Final Test MSE Loss: {avg_test_loss:.4f}")

# Plot loss curves
# Loss curve shows some signs of overfitting the small dataset
# likely contributes to this but can be mitigated using regularisation
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid(True)
plt.show()


We now need to run an inference on our trained model with the test data to obtain our predictions, and then proceed to obtain the metrics as we did with the other models for comparison. So we need to create a dictionary to store the metrics for the train, validation, and test data.

#### Task 9 - Obtain the MLP Predictions of the Test Data and Get Performance Metrics
Recall:
 - When running an inference on a trained model in PyTorch, what do you need to do?
- You scaled your dataset before the training. What should do to the model predictions before computing the performance metrics?

In [ ]:
# TODO
# what do you need to do when running an inference on a trained model in PyTorch?
# what should do to the model predictions before computing the performance metrics?

In [ ]:
loaders = prep_loaders(data=data, keys=keys, scaler_inp=scaler_inp, scaler_out=scaler_out, no_shuffle=True)

mlp_preds = {}
model.eval()

for (key, loader) in zip(keys, loaders):
    pass
    # TODO
    # 1. obtain model predictions
    # 2. you scaled your dataset before the training. What should do to the model predictions before computing the performance metrics?
    # 3. save the predictions in a dictionary for passing into the print_err function

err_mlp = None # print_err(?)

Now plot a bar chart like the ones we've plotted before, but now including the NN's performance metrics. Compare the performance of all the models.

- Comment on the performance of the MLP relative to that of DC-OPF and the linear model.
- Is this performance what you expected? Why or why not?
- What can you conclude from the results?

In [ ]:
plot_comparison_bar(errors=[err_gw, err_nw, err_dc, err_lm, err_mlp], err_key='MCEM', methods=['grid', 'node', 'dcopf', 'lm', 'mlp'])

#### End of Practical

     _______  __   __  _______  __    _  ___   _  _______  __
    |       ||  | |  ||   _   ||  |  | ||   | | ||       ||  |
    |_     _||  |_|  ||  |_|  ||   |_| ||   |_| ||  _____||  |
      |   |  |       ||       ||       ||      _|| |_____ |  |
      |   |  |       ||       ||  _    ||     |_ |_____  ||__|
      |   |  |   _   ||   _   || | |   ||    _  | _____| | __
      |___|  |__| |__||__| |__||_|  |__||___| |_||_______||__|

## Thanks!

For more information on these subjects and more you might want to check
the following resources.

-   company: [Trent AI](https://trent.ai)
-   book: [The Atomic
    Human](https://www.penguin.co.uk/books/455130/the-atomic-human-by-lawrence-neil-d/9780241625248)
-   twitter: [@lawrennd](https://twitter.com/lawrennd)
-   podcast: [The Talking Machines](http://thetalkingmachines.com)
-   newspaper: [Guardian Profile
    Page](http://www.theguardian.com/profile/neil-lawrence)
-   blog:
    [http://inverseprobability.com](http://inverseprobability.com/blog.html)